In [1]:
import numpy as np

path_prefix = "./all/data/"
word_to_index = {} # map word to index
g_word_list = [] # sorted by frequence low to high
total_word_dict = {}
skip_words = set(['', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'as', 'an', 'and', 'are', 'at', 'by', 'down', 'for', 'from', 'front', 'has', 'in', 'is', 'it', 'near', 'next', 'of', 'on', 'out', 'over', 'several', 'sits', 'some', 'that', 'the', 'there', 'through', 'to', 'top', 'up', 'while', 'with'])
train_des = [] #0/1/2 array of word
test_des = [] #0/1/2 array of word
train_feature_list = [] # list of feature_array by index
test_feature_list = []
all_predict_test_des = []
train_g_category_tag = []
train_g_item_tag = []
test_g_category_tag = []
test_g_item_tag = []
train_captions = []
test_captions = []
MAX_INPUT = 10000
MAX_OUTPUT = 2000
COMPONENTS = 400 # need to run again
PCA_NUMBER = 2000 # need to run again
ITER = 500

In [2]:
def read_file_to_string(file_name, keep_colon):
    f = open(file_name, "r")
    content = f.read()
    f.close()
    content = content.lower()
    content = content.replace(",", " ").replace(".", " ").replace("'", " ").replace("\"", " ")
    if (keep_colon == False):
        content = content.replace(":", " ")
    return content

def read_file_to_total_word_dict(file_name, keep_colon):
    word_list = read_file_to_string(file_name, keep_colon).split()
    for name in word_list:
        if name in skip_words:
            continue
        if name in total_word_dict:
            total_word_dict[name] += 1
            continue
        else:
            total_word_dict[name] = 1
            continue

def print_word_in_array(a):
    ret = ""
    for i in range(len(a)):
        if a[i] != 0:
            ret = ret + str(a[i]) + g_word_list[i] + " "
    print(ret)

In [3]:
total_word_dict = {}
for i in range(MAX_INPUT):
    read_file_to_total_word_dict(path_prefix+"descriptions_train/"+str(i)+".txt", False)
    read_file_to_total_word_dict(path_prefix+"tags_train/"+str(i)+".txt", True)
    read_file_to_total_word_dict(path_prefix+"captions_train/"+str(i)+".txt", True)

In [4]:
import functools
def cmp(a, b):
    a_num = abs(float(a.split("___")[0]))
    b_num = abs(float(b.split("___")[0]))
    if (a_num < b_num):
        return -1
    return 1

In [5]:
# load g_word_list
g_word_list = []
word_to_index = {}
tmp_list = []
#TODO: maybe try filter on number
for word in total_word_dict:
    if total_word_dict[word] <= 0:
        continue
    tmp_list.append(str(total_word_dict[word])+"___"+word)

tmp_list = sorted(tmp_list, key=functools.cmp_to_key(cmp))

for i in range(len(tmp_list)):
    word = tmp_list[i].split("___")[1]
    g_word_list.append(word)
    word_to_index[word] = i

In [6]:
# read to tags
train_g_category_tag = [0] * MAX_INPUT
train_g_item_tag = [0] * MAX_INPUT
test_g_category_tag = [0] * MAX_OUTPUT
test_g_item_tag = [0] * MAX_OUTPUT

def is_in_tag(file_number, word, train):
    if train:
        return (word in train_g_category_tag[file_number]) or (word in train_g_item_tag[file_number])
    return (word in test_g_category_tag[file_number]) or (word in test_g_item_tag[file_number])

def read_tags(file_name, train):
    lines = read_file_to_string(file_name, True).split("\n")
    index = int(file_name.split("/")[-1].replace(".txt", ""))
    current_category_set = set([])
    current_item_set = set([])
    
    for line in lines:
        cat = line.split(":")[0].strip()
        if cat in word_to_index:
            current_category_set.add(cat)

        item = ''
        if (cat.strip() != line.replace(":", "").strip()):
            item = line.split(":")[1].strip()
        if item in word_to_index:
            current_item_set.add(item)
    if (train):
        train_g_category_tag[index] = current_category_set
        train_g_item_tag[index] = current_item_set
    else:
        test_g_category_tag[index] = current_category_set
        test_g_item_tag[index] = current_item_set
    

for i in range(MAX_INPUT):
    read_tags(path_prefix+"tags_train/"+str(i)+".txt", True)
for i in range(MAX_OUTPUT):
    read_tags(path_prefix+"tags_test/"+str(i)+".txt", False)

In [7]:
train_captions = [0] * MAX_INPUT
test_captions = [0] * MAX_OUTPUT

# read captions to tags
def read_captions(file_name, train):
    lines = read_file_to_string(file_name, False).split()
    index = int(file_name.split("/")[-1].replace(".txt", ""))
    current_word_set = set([])

    for line in lines:
        word_list = line.strip().split()
        for word in word_list:
            if word in word_to_index:
                current_word_set.add(word)
    if (train):
        train_captions[index] = current_word_set
    else:
        test_captions[index] = current_word_set

for i in range(MAX_INPUT):
    read_captions(path_prefix+"captions_train/"+str(i)+".txt", True)
for i in range(MAX_OUTPUT):
    read_captions(path_prefix+"captions_test/"+str(i)+".txt", False)

In [8]:
# where to make the change
# todo: remove count to match test output
# lenpi ci, yaoshi you dehua, jianfen
# also try norm

# count_max_weight = 2.0
# uniqueness_max_weight = 2.0
# TODO: add words in tag_item to pic
def my_weight_func(index, count, train, file_num):
    if (count <= 0):
        return 0
#     count_apply = 0
#     if is_in_tag(file_num, g_word_list[index], train):
#         count_apply = 1
#     elif count >= 3:
#         count_apply = 1
#     return 1 + (len(g_word_list)-index)*uniqueness_max_weight/len(g_word_list) + count_apply*count_max_weight

#     if count >= 5:
#         return count
#     if is_in_tag(file_num, g_word_list[index], train):
#         return 5
    return 1

def weight_occurance_array(occurance_array, train, file_num):
    ret = [0] * len(occurance_array)
    for index in range(len(occurance_array)):
        ret[index] = my_weight_func(index, occurance_array[index], train, file_num)
    return ret

def read_des_to_list(path, file_num):
    ret = []
    train = (path.find("train") != -1)
    for i in range(file_num):
        int_array = [0] * len(g_word_list)
        temp_list = read_file_to_string(path+str(i)+".txt", False).split()
        if train:
            temp_list.extend(train_captions[i])
        else:
            temp_list.extend(test_captions[i])
        for word in temp_list:
            if not word in word_to_index:
                continue
            int_array[word_to_index[word]] += 1
        ret.append(weight_occurance_array(int_array, train, i))
    return ret

train_des = read_des_to_list(path_prefix+"descriptions_train/", MAX_INPUT)
test_des = read_des_to_list(path_prefix+"descriptions_test/", MAX_OUTPUT)

In [9]:
# add tag to desc list

#TODO: careful with tag number
def add_tag_to_train_des():
    for sentence_index in range(MAX_INPUT):
        array_to_add = [0] * len(g_word_list)
        for tag in train_g_item_tag[sentence_index]:
            array_to_add[word_to_index[tag]] = 1
        
        ret = train_des[sentence_index].copy()
        for word_index in range(len(g_word_list)):
            ret[word_index] = max(ret[word_index], array_to_add[word_index])
        train_des[sentence_index] = ret

def add_tag_to_test_des():
    for sentence_index in range(MAX_OUTPUT):
        array_to_add = [0] * len(g_word_list)
        for tag in test_g_item_tag[sentence_index]:
            array_to_add[word_to_index[tag]] = 1
        
        ret = all_predict_test_des[sentence_index].copy()
        print(all_predict_test_des[sentence_index])
        for word_index in range(len(g_word_list)):
            ret[word_index] = max(ret[word_index], array_to_add[word_index])
        all_predict_test_des[sentence_index] = ret


In [10]:
add_tag_to_train_des()

In [11]:
# load feature list
def load_feature_into_list(filename):
    f = open(filename, "r")
    content = f.readlines()
    f.close()
    train = filename.find("train")
    for line in content:
        name_index = int(line.split(".jpg,")[0].split("/")[1])
        feature_list = line.split(".jpg,")[1].split(",")
        if train != -1:
            if name_index >= MAX_INPUT:
                continue
            train_feature_list[name_index] = feature_list
        else:
            if name_index >= MAX_OUTPUT:
                continue
            test_feature_list[name_index] = feature_list

train_feature_list = [0] * MAX_INPUT
test_feature_list = [0] * MAX_OUTPUT
# load_feature_into_list(path_prefix+"features_train/features_resnet1000_train.csv")
# load_feature_into_list(path_prefix+"features_test/features_resnet1000_test.csv")
load_feature_into_list(path_prefix+"features_train/features_resnet1000intermediate_train.csv")
load_feature_into_list(path_prefix+"features_test/features_resnet1000intermediate_test.csv")

train_feature_list = np.asarray(train_feature_list, dtype=np.float64)
test_feature_list = np.asarray(test_feature_list, dtype=np.float64)

In [12]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components = PCA_NUMBER)
# pca.fit(train_des)
# train_des_pca = pca.transform(train_des)
# test_des_pca = pca.transform(test_des)
train_des_pca = np.asarray(train_des)
test_des_pca = np.asarray(test_des)

print(train_des_pca.shape)
print(test_des_pca.shape)

# pca2 = PCA(n_components = PCA_NUMBER)
# pca2.fit(train_feature_list)
# train_feature_list = pca2.transform(train_feature_list)
# test_feature_list = pca2.transform(test_feature_list)
train_feature_list = np.asarray(train_feature_list)
test_feature_list = np.asarray(test_feature_list)

print(train_feature_list.shape)
print(test_feature_list.shape)

from sklearn.cross_decomposition import PLSRegression
print("Done 0")
model = PLSRegression(COMPONENTS, max_iter=ITER)
print("Done 1")
model.fit(train_feature_list, train_des_pca)
print("Done 2")

(10000, 9901)
(2000, 9901)
(10000, 2048)
(2000, 2048)
Done 0
Done 1


C:\Users\ranji\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:77: UserWarning: Maximum number of iterations reached
  warnings.warn('Maximum number of iterations reached')


Done 2


In [13]:
print("Start 3")
all_predict_test_des = model.predict(test_feature_list)
print("Done 3")

Start 3
Done 3


In [14]:
add_tag_to_test_des()

[ 2.98509616e-04  2.98509616e-04  4.31875001e-04 ...  0.00000000e+00
  3.77702623e-01 -3.19856032e-02]
[0.00140041 0.00140041 0.00257581 ... 0.         0.19929253 0.26987791]
[6.11082070e-04 6.11082070e-04 1.16659374e-03 ... 0.00000000e+00
 2.40152308e-01 8.00235896e-01]
[ 0.00181648  0.00181648 -0.00104554 ...  0.          0.30578907
  0.22287202]
[ 7.18116875e-04  7.18116875e-04  6.44978658e-04 ...  0.00000000e+00
  8.83521261e-01 -3.36080670e-02]
[ 0.00098108  0.00098108 -0.00101241 ...  0.          0.19250355
 -0.0545234 ]
[0.00165095 0.00165095 0.00410868 ... 0.         0.7275168  0.14569211]
[ 5.70950835e-04  5.70950835e-04  9.98541785e-04 ...  0.00000000e+00
 -2.88169379e-02  7.87486669e-01]
[ 9.70517631e-04  9.70517631e-04 -4.51104348e-04 ...  0.00000000e+00
  6.81655539e-01  2.60919478e-02]
[ 2.65760833e-05  2.65760833e-05 -2.53769243e-03 ...  0.00000000e+00
  9.89066583e-01 -2.31165345e-02]
[ 0.00024423  0.00024423 -0.00220541 ...  0.          0.14165545
  0.14717709]
[ 2.387

  0.70862355]
[7.90178183e-05 7.90178183e-05 3.57613352e-03 ... 0.00000000e+00
 4.24128701e-01 2.97918788e-02]
[1.23332452e-04 1.23332452e-04 4.31630815e-03 ... 0.00000000e+00
 2.19761992e-01 9.38423628e-02]
[ 0.00202321  0.00202321 -0.00238527 ...  0.          0.21173561
  0.14298721]
[0.00404696 0.00404696 0.00739221 ... 0.         0.08491229 0.05529547]
[-7.12444291e-05 -7.12444291e-05 -5.22505145e-04 ...  0.00000000e+00
  2.79231929e-01  5.96884138e-03]
[-1.21408107e-04 -1.21408107e-04  1.26573370e-03 ...  0.00000000e+00
  1.72990442e-01  3.80231606e-01]
[ 0.00137617  0.00137617 -0.00102697 ...  0.          0.27106369
 -0.1486361 ]
[ 0.00058812  0.00058812 -0.00498621 ...  0.          0.35031
 -0.00258126]
[-7.22469330e-04 -7.22469330e-04 -1.39278393e-03 ...  0.00000000e+00
  1.18205009e+00  8.24885439e-01]
[ 4.18110165e-04  4.18110165e-04 -1.80796958e-03 ...  0.00000000e+00
  1.99005575e-01  1.08338503e+00]
[ 0.00212183  0.00212183 -0.00145192 ...  0.         -0.31208349
  0.61824

[-0.00137056 -0.00137056  0.00075492 ...  0.          0.52540129
  0.42046371]
[ 1.21050595e-03  1.21050595e-03 -2.15807397e-05 ...  0.00000000e+00
  5.61837242e-01 -1.18475506e-01]
[ 0.00247199  0.00247199  0.00180723 ...  0.         -0.16033565
  0.97529478]
[ 2.88273137e-04  2.88273137e-04 -6.05983230e-04 ...  0.00000000e+00
  5.40146328e-01 -9.28484925e-02]
[-0.00120676 -0.00120676 -0.00104181 ...  0.          0.3386938
  0.33231546]
[-0.00096533 -0.00096533 -0.00101575 ...  0.          0.5431402
  0.12464445]
[-1.57060255e-04 -1.57060255e-04 -1.05564756e-03 ...  0.00000000e+00
  8.12617158e-01  5.23693880e-01]
[4.29643517e-04 4.29643517e-04 3.01966294e-04 ... 0.00000000e+00
 4.48398528e-01 8.56018762e-01]
[ 0.0008303   0.0008303  -0.00118025 ...  0.          0.40063284
 -0.04882324]
[ 2.77363189e-04  2.77363189e-04 -1.11780059e-04 ...  0.00000000e+00
  2.33518572e-01  5.28541588e-03]
[-0.00193291 -0.00193291  0.00502886 ...  0.          0.24285288
 -0.2325797 ]
[8.69595352e-04 8.6

[-0.00226809 -0.00226809  0.00220302 ...  0.         -0.06544185
  0.42156179]
[-0.00049201 -0.00049201 -0.00296674 ...  0.          0.2778937
  0.4202275 ]
[ 0.00220043  0.00220043  0.00101318 ...  0.          0.13003444
 -0.10159553]
[ 0.00105889  0.00105889 -0.00156891 ...  0.          0.1378834
  0.94842877]
[2.11301959e-03 2.11301959e-03 4.23351466e-04 ... 0.00000000e+00
 7.96014090e-01 2.25773622e-01]
[ 1.55259411e-04  1.55259411e-04 -9.48184636e-04 ...  0.00000000e+00
  3.48726420e-01  5.70816014e-01]
[ 0.00457677  0.00457677 -0.00249646 ...  0.          0.71077225
 -0.09630429]
[-4.69960400e-06 -4.69960400e-06  1.75683845e-03 ...  0.00000000e+00
  4.60484510e-01  1.71288021e-01]
[-1.41896108e-03 -1.41896108e-03 -2.04248068e-04 ...  0.00000000e+00
  6.90608054e-01  1.13204935e-01]
[-0.00133511 -0.00133511  0.00072065 ...  0.          0.70701963
 -0.14727851]
[-9.07528970e-04 -9.07528970e-04  1.31090136e-04 ...  0.00000000e+00
  5.68111020e-01  2.00901174e-01]
[ 0.00318059  0.003

[-1.46114953e-04 -1.46114953e-04 -2.33458209e-03 ...  0.00000000e+00
  2.61637626e-01 -1.91752864e-03]
[ 0.00110064  0.00110064  0.00240518 ...  0.          0.61922671
 -0.01292289]
[-1.13126178e-03 -1.13126178e-03 -7.34200209e-04 ...  0.00000000e+00
  8.20956991e-02  9.65926302e-01]
[-0.00264278 -0.00264278  0.00114411 ...  0.          0.31339727
  0.6314853 ]
[ 0.00256208  0.00256208  0.00306159 ...  0.         -0.04586102
  1.10233652]
[ 0.0009068   0.0009068  -0.00231795 ...  0.          0.22372393
 -0.09858657]
[0.00205662 0.00205662 0.00123391 ... 0.         0.47789917 0.06500779]
[-0.00202273 -0.00202273 -0.00179887 ...  0.          0.2302973
 -0.31996241]
[7.70751790e-05 7.70751790e-05 5.17169949e-03 ... 0.00000000e+00
 5.13494237e-01 9.37745596e-02]
[ 0.00034879  0.00034879 -0.00050077 ...  0.          0.25051247
  0.13519821]
[0.00116087 0.00116087 0.00142662 ... 0.         0.49843333 0.81959982]
[-1.23478393e-04 -1.23478393e-04 -1.60301903e-03 ...  0.00000000e+00
  5.1878822

...

[ 2.69987264e-04  2.69987264e-04 -8.08148877e-04 ...  0.00000000e+00
  5.37550445e-01  4.83691685e-02]
[-3.74452930e-04 -3.74452930e-04  1.53175651e-03 ...  0.00000000e+00
 -3.00485498e-02  1.12874255e+00]
[0.00203478 0.00203478 0.00094648 ... 0.         0.66824035 0.03748939]
[ 0.00452218  0.00452218 -0.00232834 ...  0.          0.43531906
  0.21696633]
[-0.00271648 -0.00271648 -0.00395535 ...  0.          0.83408062
  0.91232477]
[ 1.67751934e-04  1.67751934e-04 -6.39106690e-04 ...  0.00000000e+00
  4.99778474e-01  4.85486296e-01]
[-0.00064795 -0.00064795  0.00271625 ...  0.          0.03581463
  0.15712683]
[-0.00092542 -0.00092542  0.00025639 ...  0.         -0.0234244
  0.23885494]
[-0.00076926 -0.00076926 -0.00231709 ...  0.          0.73562363
  0.50205312]
[-1.72345297e-03 -1.72345297e-03  2.68841191e-03 ...  0.00000000e+00
  7.21244222e-01 -3.57005146e-05]
[ 1.92481824e-04  1.92481824e-04 -2.00773866e-03 ...  0.00000000e+00
 -1.08593228e-01  7.45899724e-01]
[ 0.00216229  0.002

[-0.00061869 -0.00061869  0.00167899 ...  0.          0.07974159
 -0.16450435]
[1.50699726e-05 1.50699726e-05 1.15365620e-02 ... 0.00000000e+00
 2.46780076e-01 2.15911943e-01]
[-0.00042825 -0.00042825 -0.00329148 ...  0.          0.22517197
 -0.15893094]
[-4.23569155e-04 -4.23569155e-04 -2.03713134e-03 ...  0.00000000e+00
  4.46107589e-01  3.28369202e-02]
[-0.00191446 -0.00191446  0.01229413 ...  0.          0.33961564
  0.37095932]
[ 5.61374521e-04  5.61374521e-04  2.35013861e-04 ...  0.00000000e+00
  5.37469717e-01 -5.50172412e-02]
[-0.00177259 -0.00177259 -0.00157864 ...  0.          0.62937659
  0.12408047]
[ 5.99454039e-04  5.99454039e-04 -3.21316493e-04 ...  0.00000000e+00
  5.84160751e-01  3.66468337e-01]
[-0.00181831 -0.00181831 -0.00271006 ...  0.          1.10971327
  0.01028717]
[6.17736780e-04 6.17736780e-04 3.55631326e-03 ... 0.00000000e+00
 6.34811131e-01 2.35525312e-01]
[3.81550226e-03 3.81550226e-03 1.16477557e-04 ... 0.00000000e+00
 6.14675749e-01 1.94836281e-01]
[ 0.0

[-2.97118278e-03 -2.97118278e-03 -2.32089873e-04 ...  0.00000000e+00
  1.14094451e-02  2.76478763e-01]
[0.00171965 0.00171965 0.00264276 ... 0.         0.34079349 0.40854996]
[ 0.00253483  0.00253483 -0.0011226  ...  0.          0.65771975
 -0.0614166 ]
[ 0.00198323  0.00198323 -0.00071659 ...  0.          0.36574055
 -0.0339735 ]
[3.75997511e-03 3.75997511e-03 1.24183038e-04 ... 0.00000000e+00
 2.61152560e-01 2.29084350e-01]
[ 0.00194803  0.00194803 -0.00242334 ...  0.          0.02429475
  0.59191072]
[-0.0037698  -0.0037698  -0.0028169  ...  0.          0.35018608
  0.09360808]
[0.00038891 0.00038891 0.00110118 ... 0.         0.28278712 0.17800401]
[-0.00136025 -0.00136025 -0.00126871 ...  0.          0.11436277
  0.39012801]
[ 9.58939355e-04  9.58939355e-04 -6.15303508e-04 ...  0.00000000e+00
  2.63089237e-01  1.03432828e+00]
[ 4.67500676e-04  4.67500676e-04 -3.36434561e-04 ...  0.00000000e+00
  4.31927320e-01  1.73904920e-01]
[ 0.00274061  0.00274061 -0.00153533 ...  0.          0

[ 2.03844258e-03  2.03844258e-03 -7.49885140e-05 ...  0.00000000e+00
  3.10310173e-01  5.95988301e-01]
[ 1.02684804e-03  1.02684804e-03 -7.89226373e-05 ...  0.00000000e+00
  1.02724744e+00 -1.06278218e-01]
[ 0.00382097  0.00382097 -0.00608394 ...  0.          0.8562501
  0.15342999]
[ 0.00042489  0.00042489 -0.00244207 ...  0.          0.4110056
 -0.18520968]
[-0.00321221 -0.00321221 -0.00036341 ...  0.          0.004238
 -0.08306268]
[ 0.00164742  0.00164742 -0.00577279 ...  0.          0.21756136
  0.01049441]
[2.24104488e-03 2.24104488e-03 4.37756106e-04 ... 0.00000000e+00
 6.17334513e-01 2.31351628e-01]
[-0.00061763 -0.00061763 -0.00074911 ...  0.          0.4428254
 -0.04208403]
[-1.00316573e-03 -1.00316573e-03  2.54140450e-04 ...  0.00000000e+00
 -3.48464692e-03  4.39703842e-01]
[ 0.00128988  0.00128988 -0.00196485 ...  0.          0.56858646
 -0.14151687]
[ 1.88962972e-04  1.88962972e-04 -4.18306266e-03 ...  0.00000000e+00
  7.68829541e-01 -9.88366362e-02]
[-0.00146783 -0.001467

[-0.0011665  -0.0011665  -0.00113324 ...  0.          0.58304055
  0.32585061]
[-0.00053171 -0.00053171  0.0024106  ...  0.         -0.05601967
 -0.06895474]
[-0.00070999 -0.00070999 -0.00055385 ...  0.          0.09096737
  0.04335235]
[ 0.00100598  0.00100598 -0.00167575 ...  0.          0.31415477
  0.12745127]
[6.80495060e-04 6.80495060e-04 2.21623821e-04 ... 0.00000000e+00
 2.87927397e-01 6.43752001e-03]
[0.00322359 0.00322359 0.00340926 ... 0.         0.49491038 0.62504057]
[ 0.00102104  0.00102104  0.00832269 ...  0.         -0.37595715
  0.85005773]
[0.00449062 0.00449062 0.00115318 ... 0.         0.10670206 0.21017963]
[ 5.38443575e-04  5.38443575e-04 -3.47633667e-03 ...  0.00000000e+00
  5.67962787e-01  4.90107477e-01]
[1.97741493e-03 1.97741493e-03 9.11859294e-06 ... 0.00000000e+00
 7.17283780e-01 4.81130122e-01]
[-1.38272898e-04 -1.38272898e-04  1.84054873e-04 ...  0.00000000e+00
  1.62323888e-01  1.07998954e+00]
[ 0.0009925   0.0009925  -0.00252672 ...  0.         -0.01860

In [15]:
import sklearn.metrics

# return a score
def compare_feature(target, given):
    return float(sklearn.metrics.pairwise.cosine_distances([np.array(target, dtype=float)], [np.array(given, dtype=float)]))
#     return float(np.linalg.norm(np.array(target, dtype=float) - np.array(given, dtype=float)))

def get_top_20(target):
    ret = []
    for index in range(MAX_OUTPUT):
        given = all_predict_test_des[index]
        score = compare_feature(target, given)
        item = str(score) + "___" + str(index)
        ret.append(item)
    return sorted(ret, key=functools.cmp_to_key(cmp))[:20]

# f = open("reverse_count_"+str(count_max_weight)+"unique"+str(uniqueness_max_weight)+".csv", "w")
f = open("reverse_with_captions.csv", "w")
f.write("Descritpion_ID,Top_20_Image_IDs\n")
for index in range(MAX_OUTPUT):
    target_desc = test_des_pca[index]
    twenty_image_list = get_top_20(target_desc)
    string_to_write = "" + str(index) + ".txt,"
    for name in twenty_image_list:
        file_name = name.split("___")[1] + ".jpg"+" "
        string_to_write += file_name
    string_to_write = string_to_write.strip() + "\n"
    print(string_to_write)
    f.write(string_to_write)

f.close()

0.txt,575.jpg 1105.jpg 93.jpg 1869.jpg 352.jpg 1135.jpg 255.jpg 831.jpg 856.jpg 974.jpg 1420.jpg 1238.jpg 1629.jpg 381.jpg 1131.jpg 123.jpg 1702.jpg 1011.jpg 700.jpg 1811.jpg

1.txt,833.jpg 288.jpg 1.jpg 376.jpg 1156.jpg 285.jpg 589.jpg 1760.jpg 427.jpg 418.jpg 1890.jpg 1841.jpg 1469.jpg 1820.jpg 1438.jpg 314.jpg 627.jpg 1220.jpg 1279.jpg 818.jpg

2.txt,598.jpg 1724.jpg 445.jpg 1840.jpg 159.jpg 1713.jpg 1208.jpg 1030.jpg 838.jpg 332.jpg 1437.jpg 1084.jpg 649.jpg 953.jpg 1665.jpg 1421.jpg 42.jpg 1366.jpg 1651.jpg 1471.jpg

3.txt,1421.jpg 904.jpg 649.jpg 1059.jpg 1208.jpg 1475.jpg 1084.jpg 1471.jpg 1665.jpg 1366.jpg 821.jpg 763.jpg 1023.jpg 1030.jpg 1024.jpg 696.jpg 322.jpg 1330.jpg 979.jpg 1499.jpg

4.txt,95.jpg 233.jpg 1986.jpg 639.jpg 543.jpg 1238.jpg 1593.jpg 18.jpg 1299.jpg 1816.jpg 1670.jpg 672.jpg 722.jpg 1276.jpg 742.jpg 4.jpg 1498.jpg 1324.jpg 292.jpg 922.jpg

5.txt,771.jpg 1991.jpg 51.jpg 492.jpg 330.jpg 1693.jpg 814.jpg 1335.jpg 703.jpg 1874.jpg 630.jpg 846.jpg 1660.jpg 379.jp

47.txt,1133.jpg 1767.jpg 817.jpg 937.jpg 1731.jpg 441.jpg 1149.jpg 17.jpg 148.jpg 340.jpg 1536.jpg 223.jpg 1511.jpg 618.jpg 1126.jpg 1253.jpg 234.jpg 1922.jpg 1632.jpg 1719.jpg

48.txt,1713.jpg 445.jpg 953.jpg 1866.jpg 904.jpg 1665.jpg 838.jpg 60.jpg 1840.jpg 1242.jpg 159.jpg 696.jpg 1664.jpg 1641.jpg 1634.jpg 934.jpg 431.jpg 634.jpg 1724.jpg 1366.jpg

49.txt,1761.jpg 887.jpg 1888.jpg 274.jpg 1818.jpg 1830.jpg 334.jpg 554.jpg 892.jpg 1696.jpg 1895.jpg 1743.jpg 389.jpg 703.jpg 814.jpg 630.jpg 526.jpg 1136.jpg 595.jpg 1693.jpg

50.txt,932.jpg 183.jpg 1711.jpg 1534.jpg 560.jpg 1832.jpg 1190.jpg 384.jpg 1290.jpg 1432.jpg 415.jpg 1302.jpg 54.jpg 1469.jpg 1703.jpg 1210.jpg 1363.jpg 1223.jpg 1022.jpg 1362.jpg

51.txt,1727.jpg 1097.jpg 1379.jpg 1145.jpg 514.jpg 1404.jpg 475.jpg 1886.jpg 1088.jpg 1215.jpg 1818.jpg 26.jpg 1889.jpg 1207.jpg 330.jpg 354.jpg 1630.jpg 554.jpg 1700.jpg 1696.jpg

52.txt,346.jpg 498.jpg 620.jpg 1926.jpg 1746.jpg 52.jpg 914.jpg 1798.jpg 1434.jpg 921.jpg 745.jpg 1213.jpg

94.txt,1143.jpg 885.jpg 1039.jpg 1807.jpg 331.jpg 286.jpg 835.jpg 169.jpg 571.jpg 185.jpg 1116.jpg 160.jpg 1714.jpg 589.jpg 1408.jpg 1313.jpg 14.jpg 818.jpg 1134.jpg 1550.jpg

95.txt,51.jpg 1335.jpg 814.jpg 1426.jpg 492.jpg 1943.jpg 1693.jpg 771.jpg 703.jpg 630.jpg 846.jpg 20.jpg 379.jpg 1913.jpg 526.jpg 1888.jpg 1830.jpg 274.jpg 321.jpg 887.jpg

96.txt,445.jpg 1713.jpg 953.jpg 1665.jpg 1840.jpg 634.jpg 696.jpg 1499.jpg 1124.jpg 1033.jpg 1724.jpg 159.jpg 1471.jpg 1866.jpg 511.jpg 904.jpg 1716.jpg 1084.jpg 838.jpg 769.jpg

97.txt,1739.jpg 607.jpg 1664.jpg 545.jpg 690.jpg 1033.jpg 1211.jpg 445.jpg 1460.jpg 1483.jpg 938.jpg 1440.jpg 1487.jpg 319.jpg 1605.jpg 680.jpg 598.jpg 1866.jpg 180.jpg 399.jpg

98.txt,864.jpg 47.jpg 1004.jpg 899.jpg 1706.jpg 420.jpg 712.jpg 459.jpg 1821.jpg 848.jpg 1600.jpg 1179.jpg 837.jpg 1817.jpg 343.jpg 521.jpg 221.jpg 414.jpg 328.jpg 982.jpg

99.txt,1683.jpg 830.jpg 1825.jpg 1417.jpg 1267.jpg 1631.jpg 1585.jpg 1685.jpg 1576.jpg 1640.jpg 1374.jpg 215.jpg 1861.jpg

141.txt,1127.jpg 37.jpg 2.jpg 31.jpg 1433.jpg 1385.jpg 1098.jpg 734.jpg 1676.jpg 670.jpg 1736.jpg 1696.jpg 636.jpg 1050.jpg 1236.jpg 52.jpg 1577.jpg 1504.jpg 1441.jpg 1131.jpg

142.txt,890.jpg 713.jpg 251.jpg 1014.jpg 1863.jpg 1304.jpg 1951.jpg 961.jpg 301.jpg 380.jpg 423.jpg 83.jpg 1159.jpg 988.jpg 1160.jpg 142.jpg 9.jpg 517.jpg 1549.jpg 1182.jpg

143.txt,1713.jpg 445.jpg 953.jpg 1665.jpg 143.jpg 1124.jpg 1208.jpg 1366.jpg 1664.jpg 1724.jpg 1641.jpg 1033.jpg 634.jpg 1840.jpg 1686.jpg 1340.jpg 784.jpg 696.jpg 1030.jpg 1471.jpg

144.txt,1077.jpg 1099.jpg 197.jpg 1024.jpg 949.jpg 1609.jpg 1381.jpg 1174.jpg 562.jpg 601.jpg 843.jpg 1769.jpg 717.jpg 529.jpg 1880.jpg 1231.jpg 576.jpg 1526.jpg 1679.jpg 626.jpg

145.txt,692.jpg 513.jpg 1563.jpg 694.jpg 1246.jpg 1079.jpg 1652.jpg 212.jpg 145.jpg 1080.jpg 1983.jpg 278.jpg 1357.jpg 1869.jpg 780.jpg 665.jpg 1811.jpg 1537.jpg 884.jpg 715.jpg

146.txt,111.jpg 1833.jpg 1931.jpg 146.jpg 1556.jpg 1864.jpg 1674.jpg 1987.jpg 1618.jpg 561.jpg 1763.jpg 184

188.txt,1104.jpg 803.jpg 1583.jpg 1829.jpg 1766.jpg 327.jpg 447.jpg 277.jpg 1467.jpg 1604.jpg 710.jpg 477.jpg 195.jpg 472.jpg 283.jpg 241.jpg 551.jpg 1946.jpg 581.jpg 248.jpg

189.txt,802.jpg 148.jpg 1930.jpg 1632.jpg 986.jpg 709.jpg 618.jpg 925.jpg 1767.jpg 325.jpg 1403.jpg 109.jpg 85.jpg 161.jpg 1996.jpg 852.jpg 71.jpg 547.jpg 474.jpg 1091.jpg

190.txt,1744.jpg 1640.jpg 1579.jpg 1859.jpg 1288.jpg 716.jpg 884.jpg 1357.jpg 1879.jpg 1038.jpg 145.jpg 1150.jpg 122.jpg 1063.jpg 715.jpg 1831.jpg 212.jpg 190.jpg 905.jpg 1801.jpg

191.txt,2.jpg 1816.jpg 256.jpg 141.jpg 1213.jpg 1050.jpg 1087.jpg 210.jpg 849.jpg 1046.jpg 31.jpg 734.jpg 1384.jpg 1231.jpg 620.jpg 922.jpg 199.jpg 1203.jpg 534.jpg 1466.jpg

192.txt,633.jpg 290.jpg 609.jpg 1330.jpg 1143.jpg 192.jpg 1039.jpg 331.jpg 885.jpg 1807.jpg 1134.jpg 1960.jpg 571.jpg 121.jpg 224.jpg 1408.jpg 981.jpg 1346.jpg 828.jpg 568.jpg

193.txt,54.jpg 1210.jpg 1290.jpg 1363.jpg 1022.jpg 1123.jpg 1703.jpg 1832.jpg 1616.jpg 1775.jpg 1448.jpg 1711.jpg 1534

...

1812.txt,1528.jpg 183.jpg 1534.jpg 560.jpg 1223.jpg 415.jpg 384.jpg 1832.jpg 1703.jpg 10.jpg 91.jpg 1290.jpg 1362.jpg 1432.jpg 1363.jpg 1461.jpg 1190.jpg 1116.jpg 932.jpg 1313.jpg

1813.txt,1944.jpg 1458.jpg 16.jpg 262.jpg 486.jpg 1701.jpg 1540.jpg 582.jpg 897.jpg 718.jpg 1444.jpg 1855.jpg 1472.jpg 1790.jpg 1275.jpg 1933.jpg 391.jpg 996.jpg 655.jpg 1151.jpg

1814.txt,709.jpg 1403.jpg 1632.jpg 234.jpg 925.jpg 161.jpg 1814.jpg 71.jpg 618.jpg 743.jpg 1571.jpg 148.jpg 1248.jpg 1514.jpg 1511.jpg 1418.jpg 1795.jpg 1767.jpg 1821.jpg 802.jpg

1815.txt,320.jpg 1819.jpg 432.jpg 1953.jpg 458.jpg 712.jpg 414.jpg 1450.jpg 33.jpg 779.jpg 59.jpg 1125.jpg 650.jpg 1758.jpg 1655.jpg 1151.jpg 489.jpg 899.jpg 1734.jpg 1265.jpg

1816.txt,1190.jpg 1711.jpg 932.jpg 1290.jpg 560.jpg 1832.jpg 183.jpg 1534.jpg 54.jpg 1123.jpg 1703.jpg 1875.jpg 1022.jpg 1775.jpg 1432.jpg 1469.jpg 1448.jpg 91.jpg 1363.jpg 1078.jpg

1817.txt,343.jpg 1004.jpg 1179.jpg 420.jpg 982.jpg 459.jpg 848.jpg 1706.jpg 1690.jpg 1010.jpg 866.j

1858.txt,924.jpg 278.jpg 1007.jpg 578.jpg 1058.jpg 1490.jpg 1554.jpg 396.jpg 1545.jpg 1478.jpg 671.jpg 1805.jpg 1019.jpg 1964.jpg 38.jpg 952.jpg 780.jpg 556.jpg 1349.jpg 827.jpg

1859.txt,241.jpg 1604.jpg 1467.jpg 1747.jpg 327.jpg 1946.jpg 195.jpg 1104.jpg 551.jpg 1829.jpg 710.jpg 277.jpg 1666.jpg 1947.jpg 1390.jpg 477.jpg 283.jpg 404.jpg 664.jpg 558.jpg

1860.txt,243.jpg 1637.jpg 1430.jpg 203.jpg 1370.jpg 1879.jpg 1552.jpg 1423.jpg 413.jpg 1152.jpg 1568.jpg 1342.jpg 1591.jpg 1696.jpg 1486.jpg 1480.jpg 190.jpg 747.jpg 1082.jpg 1189.jpg

1861.txt,558.jpg 1829.jpg 1244.jpg 327.jpg 241.jpg 1947.jpg 195.jpg 277.jpg 472.jpg 1390.jpg 1766.jpg 1747.jpg 1104.jpg 1467.jpg 1604.jpg 1583.jpg 551.jpg 248.jpg 1884.jpg 283.jpg

1862.txt,1452.jpg 592.jpg 1799.jpg 762.jpg 84.jpg 1309.jpg 141.jpg 1316.jpg 1663.jpg 793.jpg 1626.jpg 1489.jpg 959.jpg 1466.jpg 271.jpg 956.jpg 991.jpg 1777.jpg 935.jpg 387.jpg

1863.txt,470.jpg 713.jpg 1673.jpg 1578.jpg 1951.jpg 1549.jpg 658.jpg 746.jpg 1863.jpg 1252.jpg 115

1904.txt,1753.jpg 1156.jpg 1529.jpg 1417.jpg 562.jpg 1051.jpg 8.jpg 948.jpg 1176.jpg 1907.jpg 687.jpg 772.jpg 150.jpg 1081.jpg 265.jpg 1489.jpg 1071.jpg 1675.jpg 1902.jpg 1590.jpg

1905.txt,1684.jpg 1090.jpg 1268.jpg 1626.jpg 232.jpg 793.jpg 1245.jpg 209.jpg 102.jpg 1072.jpg 84.jpg 806.jpg 935.jpg 1314.jpg 1016.jpg 524.jpg 1113.jpg 1658.jpg 1083.jpg 917.jpg

1906.txt,1918.jpg 1948.jpg 639.jpg 1622.jpg 243.jpg 162.jpg 719.jpg 772.jpg 1261.jpg 103.jpg 1568.jpg 1001.jpg 1817.jpg 1057.jpg 1752.jpg 727.jpg 272.jpg 1.jpg 1350.jpg 256.jpg

1907.txt,843.jpg 1547.jpg 192.jpg 676.jpg 157.jpg 691.jpg 55.jpg 1553.jpg 633.jpg 224.jpg 1720.jpg 568.jpg 1330.jpg 1024.jpg 943.jpg 153.jpg 981.jpg 1443.jpg 1771.jpg 1960.jpg

1908.txt,1286.jpg 50.jpg 599.jpg 93.jpg 678.jpg 723.jpg 1047.jpg 246.jpg 1410.jpg 226.jpg 1250.jpg 202.jpg 305.jpg 1521.jpg 502.jpg 698.jpg 594.jpg 92.jpg 1980.jpg 541.jpg

1909.txt,1019.jpg 499.jpg 1058.jpg 1805.jpg 827.jpg 780.jpg 491.jpg 1586.jpg 1378.jpg 706.jpg 324.jpg 1545.jpg 

1950.txt,1950.jpg 1346.jpg 1783.jpg 837.jpg 245.jpg 1557.jpg 1622.jpg 1538.jpg 1031.jpg 923.jpg 719.jpg 103.jpg 1543.jpg 394.jpg 1847.jpg 531.jpg 1397.jpg 1901.jpg 1948.jpg 1864.jpg

1951.txt,1621.jpg 128.jpg 1235.jpg 853.jpg 1012.jpg 1364.jpg 1813.jpg 64.jpg 269.jpg 544.jpg 1376.jpg 708.jpg 214.jpg 194.jpg 1473.jpg 688.jpg 439.jpg 1623.jpg 777.jpg 1401.jpg

1952.txt,67.jpg 590.jpg 1970.jpg 440.jpg 1814.jpg 1297.jpg 439.jpg 1990.jpg 1623.jpg 673.jpg 544.jpg 1688.jpg 87.jpg 64.jpg 1936.jpg 1235.jpg 1376.jpg 1364.jpg 268.jpg 171.jpg

1953.txt,544.jpg 708.jpg 1012.jpg 1364.jpg 1621.jpg 1813.jpg 269.jpg 128.jpg 1984.jpg 1473.jpg 1623.jpg 1970.jpg 439.jpg 194.jpg 688.jpg 853.jpg 1297.jpg 777.jpg 64.jpg 1235.jpg

1954.txt,29.jpg 531.jpg 1937.jpg 1171.jpg 464.jpg 859.jpg 1200.jpg 1847.jpg 845.jpg 727.jpg 1526.jpg 1852.jpg 926.jpg 653.jpg 1497.jpg 1280.jpg 118.jpg 1037.jpg 1653.jpg 497.jpg

1955.txt,963.jpg 1863.jpg 988.jpg 95.jpg 658.jpg 1159.jpg 1578.jpg 470.jpg 746.jpg 301.jpg 890.jpg 1252.

1996.txt,748.jpg 1853.jpg 1164.jpg 319.jpg 398.jpg 860.jpg 180.jpg 938.jpg 1027.jpg 184.jpg 399.jpg 1508.jpg 680.jpg 1962.jpg 1739.jpg 1605.jpg 1460.jpg 646.jpg 1211.jpg 1008.jpg

1997.txt,104.jpg 657.jpg 1026.jpg 1163.jpg 293.jpg 58.jpg 1638.jpg 1266.jpg 782.jpg 1564.jpg 1993.jpg 1773.jpg 403.jpg 1198.jpg 1555.jpg 961.jpg 284.jpg 1061.jpg 1407.jpg 11.jpg

1998.txt,1998.jpg 625.jpg 662.jpg 257.jpg 302.jpg 303.jpg 1464.jpg 1909.jpg 1906.jpg 483.jpg 272.jpg 1338.jpg 993.jpg 1187.jpg 1752.jpg 1069.jpg 1493.jpg 790.jpg 437.jpg 675.jpg

1999.txt,405.jpg 307.jpg 1827.jpg 453.jpg 1509.jpg 729.jpg 1137.jpg 1106.jpg 1728.jpg 1342.jpg 287.jpg 407.jpg 211.jpg 772.jpg 243.jpg 1017.jpg 32.jpg 1329.jpg 1167.jpg 745.jpg

